In [10]:
import pandas as pd
df = pd.read_csv(r'/kaggle/input/movielens-20m-dataset/rating.csv')

In [11]:
df

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [12]:
user_count = df['userId'].value_counts()

In [13]:
mature_user_id = user_count[user_count > 50].index

In [14]:
df_subset = df[df['userId'].isin(mature_user_id)]

In [15]:
movie_ids = df_subset['movieId'].unique()
movies = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
movies_subset = movies[movies['movieId'].isin(movie_ids)]

In [16]:
len(movies_subset)

26701

In [17]:
movies_subset

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


## GET MOVIE DETAILS


In [18]:
import requests
def get_movie_details(tmdb_id, api_key):
    url = f"https://api.themoviedb.org/3/movie/{tmdb_id}"
    params = {
        'api_key': api_key,
        'append_to_response': 'credits'  # This gives us cast & crew
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code != 200:
            return None

        data = response.json()

        cast = data['credits']['cast']
        top_actors = [person['name'] for person in sorted(cast, key=lambda x: x['order'])[:3]]
        

        return {
            'movie_id':tmdb_id,
            'title': data['title'],
            'plot': data['overview'] or "",
            'poster_path': data['poster_path'],
            'genres': [g['name'] for g in data['genres']],
            'director': next(
                (c['name'] for c in data['credits']['crew'] if c['job'] == 'Director'),
                "Unknown"
            ),
            'top_actors': top_actors,  # ← NEW FIELD
            'release_date': data['release_date'],
            'vote_average': data['vote_average'],
            'runtime': data['runtime']
        }
    except Exception as e:
        print(f"Exception for movie {tmdb_id}: {e}")
        return None

In [19]:
details = []
api_key = "0b3c0cd034c66b8ba26c3b239e2d3354"
for i in movies_subset['movieId']:
    details.append(get_movie_details(tmdb_id=i,api_key=api_key))
    
    

In [20]:
len(details)

26701

In [21]:
details_clean = [d for d in details if d is not None]


df = pd.DataFrame(details_clean)

In [22]:
df

,movie_id,title,plot,poster_path,genres,director,top_actors,release_date,vote_average,runtime
0,2,Ariel,A Finnish man goes to the city to find a job a...,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,"[Comedy, Drama, Romance, Crime]",Aki Kaurismäki,"[Turo Pajala, Susanna Haavisto, Matti Pellonpää]",1988-10-21,7.100,73
1,3,Shadows in Paradise,"Nikander, a rubbish collector and would-be ent...",/nj01hspawPof0mJmlgfjuLyJuRN.jpg,"[Comedy, Drama, Romance]",Aki Kaurismäki,"[Matti Pellonpää, Kati Outinen, Sakari Kuosmanen]",1986-10-17,7.285,74
2,5,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,[Comedy],Robert Rodriguez,"[Tim Roth, Jennifer Beals, Antonio Banderas]",1995-12-09,5.868,98
3,6,Judgment Night,"Four young friends, while taking a shortcut en...",/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,"[Action, Crime, Thriller]",Stephen Hopkins,"[Emilio Estevez, Cuba Gooding Jr., Denis Leary]",1993-10-15,6.461,109
4,8,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,/7ln81BRnPR2wqxuITZxEciCe1lc.jpg,[Documentary],Timo Novotny,[],2006-01-01,7.150,80
...,...,...,...,...,...,...,...,...,...,...
16064,131170,Ninjaken: The Naked Sword,"In the time of warring states, the female ninj...",/3VedmUBq4Du0Py4Ezhfsi5dapYp.jpg,[Action],Yoshikazu Katō,"[Kaede Matsushima, Hiromitsu Suzuki, Kyôko Kaz...",2006-01-01,1.000,87
16065,131172,Nasty Habits,"In a Philadelphia convent, two nuns battle it ...",/7x42brVq7m4x0Rz8umBVVKuJnaU.jpg,[Comedy],Michael Lindsay-Hogg,"[Glenda Jackson, Melina Mercouri, Geraldine Page]",1977-01-28,4.600,96
16066,131174,Papst Johannes Paul II. - Das Gewissen der Welt,,None,[],Unknown,[],,0.000,0
16067,131176,White Sister,A strange love story between a devoted young n...,/cSAMynWisADWQOFeokodvsCbIrA.jpg,[Drama],Alberto Lattuada,"[Sophia Loren, Adriano Celentano, Fernando Rey]",1972-03-31,6.300,96


In [23]:
df.to_csv('/kaggle/working/curated_data.csv', index=False)